In [ ]:
##### Imports
import sys
import pandas as pd
import allel

from scripts import functions

In [ ]:
##### Constants
WORKING_DIR = ''
REF_GENOME_DIR = 'data/hg38.gencode.v41.primary_genome_by_chromosome'
REF_ANNOTATIONS_FILE = 'data/hg38.gencode.v41.primary_annotation.gtf'

VTWO_INSERTS_FILE_BASE = WORKING_DIR + 'data/gnomAD_v2.1.1/insert_fastas/gnomAD_v2.1.1_inserts'
VTWO_FIELDS = ["variants/ID", "variants/POS", "variants/ALT", "variants/FILTER_PASS", "variants/allele_type"]
VTWO_OUT_COLUMNS = ["Position", "ID", "Insert_sequence"]

VTHREE_INSERTS_FILE_BASE = WORKING_DIR + 'data/gnomADv3.1.1/insert_fastas/gnomAD_v3.1.1_inserts'
VTHREE_FIELDS = ["variants/POS", "variants/ID", "variants/ALT", "variants/FILTER_PASS", "variants/allele_type"]
VTHREE_OUT_COLUMNS = ["Position", "ID", "Insert_sequence"]

# CHUNK_LENGTH = 65_536 		# The default
CHUNK_LENGTH = 50_000

In [ ]:
##### Settings
pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows', 200)

In [ ]:
#####
# chrom = "1"
chrom = sys.argv[1]
print(chrom)

# vtwo_vcf_file = f"/datasets2/genomes/gnomAD/gnomad.genomes.r2.1.1.sites.{chrom}.vcf.bgz"
# vtwo_outfile = VTWO_INSERTS_FILE_BASE + "_chr" + chrom + ".tsv"
# vtwo_varcount = functions.file_linecount(vtwo_vcf_file, True) - len(allel.read_vcf_headers(vtwo_vcf_file)[0])
# _, _, _, vtwo_iter = allel.iter_vcf_chunks(vtwo_vcf_file, fields=VTWO_FIELDS, chunk_length=CHUNK_LENGTH)

vthree_vcf_file = f"/data/gnomAD/gnomad.genomes.v3.1.1.sites.chr{chrom}.vcf.bgz"
vthree_outfile = VTHREE_INSERTS_FILE_BASE + "_chr" + chrom + ".fa"
vthree_varcount = functions.file_linecount(vthree_vcf_file, True) - len(allel.read_vcf_headers(vthree_vcf_file)[0])
_, _, _, vthree_iter = allel.iter_vcf_chunks(vthree_vcf_file, fields=VTHREE_FIELDS, chunk_length=CHUNK_LENGTH)

In [ ]:
# ##### 
# print(len(allel.read_vcf_headers(vthree_vcf_file)[0]))
# for h in allel.read_vcf_headers(vthree_vcf_file)[0]:
# 	print(h)

In [ ]:
# ##### 
# with open(vtwo_outfile, 'w') as w:
# 	w.write("\t".join(VTWO_OUT_COLUMNS) + "\n")
	
# i = 0
# for chunk, varcount, _, _ in vtwo_iter:
# 	chunk["variants/ALT"] = [a[0][1:] for a in chunk["variants/ALT"]]		# ['AAGTCAGTCAG', '', ''] -> 'AAGTCAGTCAG' -> 'AGTCAGTCAG' because the first base is from the reference sequence
# 	chunk["variants/allele_type"] = [a[0] for a in chunk["variants/allele_type"]]
# 	df = pd.DataFrame(chunk)
# 	df.columns = [field.split("/")[1] for field in VTWO_FIELDS]
# 	df = df[(df["FILTER_PASS"]) & (df["allele_type"] == 'ins') & (df.ALT.str.len() >= 50)]
# 	df = df[["ID", "POS", "ALT"]]
# 	df.to_csv(vtwo_outfile, mode="a", index=False, header=False, sep="\t")

# 	i += varcount
# 	print(f"{i:,} of {vtwo_varcount:,}")

In [ ]:
##### 


In [ ]:
#####
with open(vthree_outfile, 'w') as _:
	pass

In [ ]:
##### 
i = 0
insert_num = 0
for chunk, varcount, _, _ in vthree_iter:
	entries = []

	chunk["variants/ALT"] = [a[0][1:] for a in chunk["variants/ALT"]]		# ['AAGTCAGTCAG', '', ''] -> 'AAGTCAGTCAG' -> 'AGTCAGTCAG' because the first base is from the reference sequence
	chunk["variants/allele_type"] = [a[0] for a in chunk["variants/allele_type"]]
	df = pd.DataFrame(chunk)
	df.columns = [field.split("/")[1] for field in VTHREE_FIELDS]
	df = df[(df["FILTER_PASS"]) & (df["allele_type"] == 'i') & (df.ALT.str.len() >= 50)]
	# df = df[["ID", "POS", "ALT"]]

	i += varcount
	print(f"{i:,} of {vthree_varcount:,}")
	
	for _, row in df.iterrows():
		insert_num += 1
		entries.append(f">chr{chrom}|{row.ID}|{row.POS}|insert_{insert_num}\n{row.ALT}\n")
	
	with open(vthree_outfile, 'a') as a:
		for entry in entries:
			a.write(entry)

In [ ]:
##### 


In [ ]:
##### 


In [ ]:
##### 


In [ ]:
##### 


#####	Discarded code	#####

# for i in range(1,23):
# 	inserts = pd.read_csv(f"data/gnomAD_2.2.1/insert_tsvs/gnomAD_v2.1.1_inserts_chr{i}.tsv", delimiter="\t")

# 	with open(f"data/gnomAD_2.2.1/insert_fastas/gnomAD_v2.1.1_inserts_chr{i}.fa", "w") as w:
# 		with open(f"data/gnomAD_v2.1.1/insert_fastas/gnomAD_v2.1.1_inserts_chr{i}_revcomp.fa", "w") as rev_comp:
# 			for _, row in inserts.iterrows():
# 				w.write(f'>{row["ID"]}|{row["Position"]}\n')
# 				w.write(f'{row["Insert_sequence"]}\n')

# 				rev_comp.write(f'>{row["ID"]}|{row["Position"]}|Reverse complement\n')
# 				rev_comp.write(f'{Seq.reverse_complement(row["Insert_sequence"])}\n')

# inserts = pd.read_csv("data/gnomAD_v2.1.1/insert_tsvs/gnomAD_v2.1.1_inserts_chrX.tsv", delimiter="\t")

# with open(f"data/gnomAD_v2.1.1/insert_fastas/gnomAD_v2.1.1_inserts_chrX.fa", "w") as w:
# 	with open(f"data/gnomAD_v2.1.1/insert_fastas/gnomAD_v2.1.1_inserts_chrX_revcomp.fa", "w") as rev_comp:
# 		for _, row in inserts.iterrows():
# 			w.write(f'>{row["ID"]}|{row["Position"]}\n')
# 			w.write(f'{row["Insert_sequence"]}\n')

# 			rev_comp.write(f'>{row["ID"]}|{row["Position"]}|Reverse complement\n')
# 			rev_comp.write(f'{Seq.reverse_complement(row["Insert_sequence"])}\n')

_, _, _, iter = allel.iter_vcf_chunks(vtwo_vcf_file, fields=["variants/REF", "variants/FILTER_PASS", "allele_type"], chunk_length=CHUNK_LENGTH)
i = 0
for chunk, varcount, _, _ in iter:
	chunk["variants/allele_type"] = [a[0] for a in chunk["variants/allele_type"]]
	df = pd.DataFrame(chunk)
	df = df[(df["variants/FILTER_PASS"]) & (df["variants/allele_type"] == 'ins')]
	assert len( df[df["variants/REF"].str.len()!=1] ) == 0, df

	i += varcount
	print(f"{i:,} of {vtwo_varcount:,}")